In [40]:
# import packages
import random
import requests
from selenium import webdriver
import time
from bs4 import BeautifulSoup

# for whatever reason, selenium gets to the end of a page, and keeps trying to click buttons
# need to try except this error away
from selenium.common.exceptions import StaleElementReferenceException

# import proxy drivers
from selenium.webdriver.common.proxy import Proxy, ProxyType

# import async packages
import asyncio
from proxybroker import Broker
import nest_asyncio
nest_asyncio.apply()

/usr/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
# function to get "N_PROXIES" proxies
def get_proxies(N_PROXIES):
    # initiate proxy list
    proxy_list = []
    
    # define async function to get proxies
    async def show(proxies):
        while True:
            proxy = await proxies.get()
            if proxy is None: break
            print('Found proxy: %s' % proxy)
            proxy_list.append(proxy)
    
    # create async loop
    proxies = asyncio.Queue()
    broker = Broker(proxies)
    tasks = asyncio.gather(
        broker.find(types=['HTTPS'], limit=N_PROXIES),
        show(proxies))
    
    # run async
    loop = asyncio.get_event_loop()
    loop.run_until_complete(tasks)
    
    return proxy_list

In [57]:
# function to test a proxy "TESTS" times
def test_proxy(HOST, PORT, TESTS):
    url = "https://www.rottentomatoes.com"
    
    # initialize proxy settings
    PROXY = str(HOST) + ":" + str(PORT)
    webdriver.DesiredCapabilities.FIREFOX['proxy'] = {
        "httpProxy": PROXY,
        "ftpProxy": PROXY,
        "sslProxy": PROXY,
        "proxyType": "MANUAL",
    }
    
    # define iter variable, using a flag technique
    ITER = 1
    
    # try to get url, if fail, increase iter
    while ITER <= TESTS:
        try:
            # get the url
            driver = webdriver.Firefox()
            driver.set_page_load_timeout(90)
            driver.get(url)
            
            # let the page load
            time.sleep(10)
            
            # look for ratings to double check if loaded
            ratings = driver.find_elements_by_class_name('dynamic-text-list__tomatometer-group')
            driver.quit()
            
            # if successful, set flag to 10, otherwise try again
            if ratings:
                ITER = 10
            else:
                ITER += 1
        
        # failure, increase iter
        except:
            print("failed " + str(ITER) + " time(s)")
            ITER += 1
            driver.quit()
    
    # if success, return True, otherwise false
    if ITER == 10:
        return True
    else:
        return False

In [43]:
def cycle_proxy(proxies):
    # make a copy of the list
    proxies_copy = proxies
    
    # use the flag method
    flag = 0
    while flag == 0:
        # choose random index
        rand_ind = random.randrange(len(proxies))
        proxy = proxies_copy[rand_ind]
        
        # test this proxy
        if test_proxy(proxy.host, proxy.port, 2):
            # if successful, return this proxy
            flag = 1
            return (proxy, proxies_copy)
        
        # otherwise, remove it
        else:
            del proxies_copy[rand_ind]

In [60]:
# get_links gets all links for movies of genre "genre" with score between "score_min" and "score_max"
# input "score_min", "score_max", and "genre" as integers
def get_links(score_min, score_max, genre, HOST, PORT):

    # create url to scrape links from
    url = "https://www.rottentomatoes.com/browse/dvd-streaming-all?" + \
        "minTomato=" + str(score_min) + "&maxTomato=" + str(score_max) + \
        "&services=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;fandango_now&genres=" + \
        str(genre) + "&sortBy=release"

    # initialize empty list to put URLs in
    endings = []
    
    # initialize proxy settings
    PROXY = str(HOST) + ":" + str(PORT)
    webdriver.DesiredCapabilities.FIREFOX['proxy'] = {
        "httpProxy": PROXY,
        "ftpProxy": PROXY,
        "sslProxy": PROXY,
        "proxyType": "MANUAL",
    }

    # open the URL
    driver = webdriver.Firefox()
    driver.set_page_load_timeout(180)
    driver.get(url)
    
    # look for button to load all the movies
    buttons = driver.find_elements_by_class_name('btn.btn-secondary-rt.mb-load-btn')
    
    # wait ten seconds for the page to load
    time.sleep(10)
    
    # while there are buttons to click, keep clicking them
    while(buttons):
        try:
            buttons[0].click()
            
            # look for more buttons to click
            buttons = driver.find_elements_by_class_name('btn.btn-secondary-rt.mb-load-btn')
            time.sleep(2)
        except StaleElementReferenceException:
            # once we reach bottom of page, break
            print("Reached bottom of page, scraping links")
            break
    
    # look for the boxes containing info about the movies
    infos = driver.find_elements_by_class_name('movie_info')
    for info in infos:
        # convert to beautiful soup objects
        soup = BeautifulSoup(info.get_attribute('innerHTML'), "html.parser")
        for link in soup.findAll('a'):
            # extract hrefs
            endings.append(link.get('href'))

    # close the browser once done
    driver.quit()
    
    return endings

/usr/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [61]:
N_PROXIES = 40
genres = [4, 5, 6, 8, 9, 10, 11, 13, 18, 14]

In [64]:
proxies = get_proxies(N_PROXIES)

/usr/lib/python3.9/site-packages/proxybroker/api.py:97: DeprecationWarning: The loop argument is deprecated since Python 3.8, and scheduled for removal in Python 3.10.
  self._on_check = asyncio.Queue(maxsize=max_conn, loop=self._loop)
/usr/lib/python3.9/asyncio/queues.py:48: DeprecationWarning: The loop argument is deprecated since Python 3.8, and scheduled for removal in Python 3.10.
  self._finished = locks.Event(loop=loop)


RuntimeError: Could not get the external IP

In [ ]:
for genre in genres:
    (proxy, proxies) = cycle_proxy(proxies)
    for i in range(8):
        score_min = i * 13
        score_max = min((i + 1) * 13 - 1, 100)
        new_links = get_links(score_min, score_max, genre, proxy.host, proxy.port)
        
        if new_links:
            links_list.append(new_links)
        else:
            print("failed " + str(score_min) + ":" + str(score_max) + " of " + str(genre))

/usr/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Reached bottom of page, scraping links
Reached bottom of page, scraping links
